<a href="https://colab.research.google.com/github/pratpat/psl_hw/blob/master/HW2_patwari3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import itertools
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [0]:
#Load boston data
df_boston = pd.read_csv('https://maya-jha.github.io/data/BostonData.csv')

#drop columns for x and y as mentioned 
#x = np.array(df_boston.drop(['Unnamed: 0','town','medv','tract'], axis = 1))
X = df_boston.drop(['Unnamed: 0','town','medv','tract'], axis = 1)
Y = df_boston['cmedv']




In [0]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(Y,X[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(X[list(feature_set)]) - Y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [0]:
def getBest(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    # Return the best model, along with some other useful information about the model
    return best_model

In [23]:
# Could take quite awhile to complete...

models_best = pd.DataFrame(columns=["RSS", "model"])
tic = time.time()
for i in range(1,15):
    models_best.loc[i] = getBest(i)
toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  


Processed 16 models on 1 predictors in 0.05647134780883789 seconds.
Processed 120 models on 2 predictors in 0.37845826148986816 seconds.
Processed 560 models on 3 predictors in 1.774949550628662 seconds.
Processed 1820 models on 4 predictors in 6.095128536224365 seconds.
Processed 4368 models on 5 predictors in 14.42930293083191 seconds.
Processed 8008 models on 6 predictors in 27.50207543373108 seconds.
Processed 11440 models on 7 predictors in 40.21273446083069 seconds.
Processed 12870 models on 8 predictors in 45.65181589126587 seconds.
Processed 11440 models on 9 predictors in 41.31657791137695 seconds.
Processed 8008 models on 10 predictors in 29.587748289108276 seconds.
Processed 4368 models on 11 predictors in 16.334585905075073 seconds.
Processed 1820 models on 12 predictors in 6.9027745723724365 seconds.
Processed 560 models on 13 predictors in 2.117331027984619 seconds.
Processed 120 models on 14 predictors in 0.4823014736175537 seconds.
Total elapsed time: 234.68003487586975

In [24]:
print(models_best.loc[2, "model"].summary())
models_best.loc[2, "model"].rsquared


                                 OLS Regression Results                                
Dep. Variable:                  cmedv   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          2.154e+35
Date:                Tue, 17 Sep 2019   Prob (F-statistic):                        0.00
Time:                        01:44:52   Log-Likelihood:                          16852.
No. Observations:                 506   AIC:                                 -3.370e+04
Df Residuals:                     504   BIC:                                 -3.369e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

1.0

In [0]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [26]:
#Load boston data
df_boston = pd.read_csv('https://maya-jha.github.io/data/BostonData.csv')

#drop columns for x and y as mentioned 
#x = np.array(df_boston.drop(['Unnamed: 0','town','medv','tract'], axis = 1))
X = df_boston.drop(['Unnamed: 0','town','medv','tract'], axis = 1)
Y = df_boston['cmedv']

models_fwd = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
predictors = []

for i in range(1,len(X.columns)+1):    
    models_fwd.loc[i] = forward(predictors)
    predictors = models_fwd.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


Processed  16 models on 1 predictors in 0.047209739685058594 seconds.
Processed  15 models on 2 predictors in 0.047189950942993164 seconds.
Processed  14 models on 3 predictors in 0.042414188385009766 seconds.
Processed  13 models on 4 predictors in 0.04049563407897949 seconds.
Processed  12 models on 5 predictors in 0.039809465408325195 seconds.
Processed  11 models on 6 predictors in 0.04257059097290039 seconds.
Processed  10 models on 7 predictors in 0.03715348243713379 seconds.
Processed  9 models on 8 predictors in 0.03136134147644043 seconds.
Processed  8 models on 9 predictors in 0.029867172241210938 seconds.
Processed  7 models on 10 predictors in 0.028743505477905273 seconds.
Processed  6 models on 11 predictors in 0.023900747299194336 seconds.
Processed  5 models on 12 predictors in 0.020357847213745117 seconds.
Processed  4 models on 13 predictors in 0.023390769958496094 seconds.
Processed  3 models on 14 predictors in 0.012803792953491211 seconds.
Processed  2 models on 15 

In [27]:
print(models_fwd.loc[1, "model"].summary())
print(models_fwd.loc[2, "model"].summary())

                                 OLS Regression Results                                
Dep. Variable:                  cmedv   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                                inf
Date:                Tue, 17 Sep 2019   Prob (F-statistic):                        0.00
Time:                        01:45:02   Log-Likelihood:                             inf
No. Observations:                 506   AIC:                                       -inf
Df Residuals:                     505   BIC:                                       -inf
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1708: RuntimeWarning: divide by zero encountered in double_scalars
  return self.mse_model/self.mse_resid
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:867: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/usr/local/lib/python3.6/dist-packages/statsmodels/stats/stattools.py:46: RuntimeWarning: invalid value encountered in double_scalars
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: divide by zero encountered in true_divide
  return self.params / self.bse


In [0]:
def backward(predictors):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [29]:
models_bwd = pd.DataFrame(columns=["RSS", "model"], index = range(1,len(X.columns)))

tic = time.time()
predictors = X.columns

while(len(predictors) > 1):  
    models_bwd.loc[len(predictors)-1] = backward(predictors)
    predictors = models_bwd.loc[len(predictors)-1]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  


Processed  16 models on 15 predictors in 0.0722804069519043 seconds.
Processed  15 models on 14 predictors in 0.060704708099365234 seconds.
Processed  14 models on 13 predictors in 0.05505514144897461 seconds.
Processed  13 models on 12 predictors in 0.0499880313873291 seconds.
Processed  12 models on 11 predictors in 0.05563235282897949 seconds.
Processed  11 models on 10 predictors in 0.0453948974609375 seconds.
Processed  10 models on 9 predictors in 0.0368800163269043 seconds.
Processed  9 models on 8 predictors in 0.034314632415771484 seconds.
Processed  8 models on 7 predictors in 0.02876114845275879 seconds.
Processed  7 models on 6 predictors in 0.02539992332458496 seconds.
Processed  6 models on 5 predictors in 0.02201390266418457 seconds.
Processed  5 models on 4 predictors in 0.02033209800720215 seconds.
Processed  4 models on 3 predictors in 0.014198780059814453 seconds.
Processed  3 models on 2 predictors in 0.010810613632202148 seconds.
Processed  2 models on 1 predictors

In [30]:
print(models_bwd.loc[1, "model"].summary())
print(models_bwd.loc[2, "model"].summary())

                                 OLS Regression Results                                
Dep. Variable:                  cmedv   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                                inf
Date:                Tue, 17 Sep 2019   Prob (F-statistic):                        0.00
Time:                        01:45:11   Log-Likelihood:                             inf
No. Observations:                 506   AIC:                                       -inf
Df Residuals:                     505   BIC:                                       -inf
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1708: RuntimeWarning: divide by zero encountered in double_scalars
  return self.mse_model/self.mse_resid
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:867: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/usr/local/lib/python3.6/dist-packages/statsmodels/stats/stattools.py:46: RuntimeWarning: invalid value encountered in double_scalars
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: divide by zero encountered in true_divide
  return self.params / self.bse


In [31]:
print("------------")
print("Best Subset:")
print("------------")
print(models_best.loc[14, "model"].params)

print("-----------------")
print("Foward Selection:")
print("-----------------")
print(models_fwd.loc[14, "model"].params)

print("-------------------")
print("Backward Selection:")
print("-------------------")
print(models_bwd.loc[14, "model"].params)

------------
Best Subset:
------------
lon      9.436896e-15
lat      1.565414e-14
cmedv    1.000000e+00
crim     6.006480e-17
zn      -1.318390e-16
indus   -2.515349e-17
chas     5.273559e-15
nox      2.074729e-14
rm      -1.026956e-15
age     -5.464379e-17
dis     -9.714451e-16
rad      5.308254e-16
b        3.523657e-17
lstat   -3.122502e-16
dtype: float64
-----------------
Foward Selection:
-----------------
cmedv      1.000000e+00
ptratio    2.636780e-16
crim      -2.883978e-16
dis        3.191891e-16
zn        -3.009745e-16
indus      1.999269e-16
nox       -7.993606e-15
age       -5.702903e-16
rad        1.561251e-16
lstat      9.020562e-17
tax       -9.584347e-17
rm         1.457168e-15
chas      -1.526557e-15
lat        5.655199e-16
dtype: float64
-------------------
Backward Selection:
-------------------
lon       -7.285839e-16
cmedv      1.000000e+00
crim      -1.354819e-15
zn         1.058181e-16
chas       6.966649e-15
nox       -1.976197e-14
rm         3.885781e-16
age  